If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syste

In [2]:
!pip install peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [4]:
 !apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [5]:
import transformers

print(transformers.__version__)

4.47.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [6]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/masked_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [7]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
datasets = load_dataset("ysfgncr/stockstimate-aapl")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

stock_data_train.jsonl:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/459 [00:00<?, ? examples/s]

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [8]:
datasets["train"][10]

{'text': "User: Here is stock data for AAPL. Last 10 days price changes: [-0.54, 0.04, 1.92, 2.35, 1.01, -0.47, 1.48, 1.37, -2.01, 0.9]%. Volume changes: [9.47, -16.35, 37.65, 1.92, -18.74, -0.96, -17.77, 30.41, -9.27, 2.9]%. RSI values: [59.53, 69.85, 68.54, 72.18, 90.12, 86.66, 93.24, 92.32, 79.65, 80.3]. MFI values: [53.6, 54.78, 55.17, 62.81, 72.04, 73.13, 80.45, 80.3, 73.16, 72.51]. Recent news: [{'headline': 'Here’s Why You Should Consider Investing in Apple (AAPL)', 'days_ago': 11, 'snippet': 'Jan 20, 2023\\xa0· Apple Inc. (NASDAQ:AAPL) delivered a 4.11% return since the beginning of the year, while its 12-month returns are down by -17.77%. The stock closed at $135.27 per share …'}, {'headline': 'Why Apple (AAPL) is Poised to Beat Earnings Estimates Again', 'days_ago': 8, 'snippet': 'Jan 23, 2023\\xa0· Apple (AAPL) has an impressive earnings surprise history and currently possesses the right combination of the two key ingredients for a likely beat in its next quarterly report.'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [11]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [14]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/459 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1098 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1305 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1092 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1368 > 1024). Running this sequence through the model will result in indexing errors


If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [15]:
tokenized_datasets["train"][1]

{'input_ids': [12982,
  25,
  3423,
  318,
  4283,
  1366,
  329,
  31518,
  43,
  13,
  4586,
  838,
  1528,
  2756,
  2458,
  25,
  685,
  16,
  13,
  3070,
  11,
  532,
  16,
  13,
  3312,
  11,
  513,
  13,
  3104,
  11,
  657,
  13,
  3901,
  11,
  657,
  13,
  2231,
  11,
  362,
  13,
  1157,
  11,
  532,
  15,
  13,
  3312,
  11,
  352,
  13,
  486,
  11,
  657,
  13,
  3459,
  11,
  532,
  15,
  13,
  4051,
  60,
  7225,
  14701,
  2458,
  25,
  25915,
  1238,
  13,
  4309,
  11,
  532,
  24,
  13,
  1314,
  11,
  807,
  13,
  2670,
  11,
  532,
  1129,
  13,
  2091,
  11,
  532,
  24,
  13,
  4524,
  11,
  807,
  13,
  4869,
  11,
  362,
  13,
  3324,
  11,
  532,
  1129,
  13,
  486,
  11,
  838,
  13,
  16,
  11,
  860,
  13,
  2857,
  60,
  7225,
  371,
  11584,
  3815,
  25,
  685,
  1954,
  13,
  1959,
  11,
  2242,
  13,
  24,
  11,
  5014,
  13,
  2920,
  11,
  5946,
  13,
  3070,
  11,
  6298,
  13,
  1731,
  11,
  6885,
  13,
  5705,
  11,
  6337,
  13,
  4309,
  11,


Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [16]:
# ✅ Ensure the tokenizer has a pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS as padding token (common for causal LM)

# ✅ Tokenize dataset properly with dynamic length
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,  # Truncate if necessary
        padding=False,  # 🚀 No fixed padding; keeps dynamic lengths
        max_length=2048  # Ensure it doesn't exceed the model limit
    )

# ✅ Tokenize dataset
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# ✅ Inspect a tokenized example
print(tokenizer.decode(tokenized_datasets["train"][2]["input_ids"]))


Map:   0%|          | 0/459 [00:00<?, ? examples/s]

User: Here is stock data for AAPL. Last 10 days price changes: [-1.06, 3.68, 0.41, 0.45, 2.11, -0.06, 1.01, 0.88, -0.54, 0.04]%. Volume changes: [-9.15, 8.39, -19.33, -9.74, 8.71, 2.77, -19.01, 10.1, 9.47, -16.35]%. RSI values: [23.9, 39.49, 43.03, 47.24, 51.84, 46.52, 54.61, 57.21, 59.53, 69.85]. MFI values: [13.23, 20.49, 28.54, 28.99, 35.6, 34.64, 40.85, 46.98, 53.6, 54.78]. Recent news: [{'headline': 'Apple Inc. (AAPL) is Attracting Investor Attention: Here is What …', 'days_ago': 9, 'snippet': 'Jan 10, 2023\xa0· Apple is expected to post earnings of $1.93 per share for the current quarter, representing a year-over-year change of -8.1%. Over the last 30 days, the Zacks Consensus …'}]. What is the expected price change in 3 days?

Assistant: <think>Analyzing AAPL: Over the last 10 days, the price trend has been upward. Volume has been decreasing. RSI is 69.85, indicating neutral conditions. MFI is 54.78, suggesting neutral market behavior. Recent news headlines: 'Apple Inc. (AAPL) i

In [17]:
from datasets import Dataset, DatasetDict

# ✅ Split dataset sequentially (First 90% for train, last 10% for test)
total_rows = len(tokenized_datasets["train"])  # Get total number of rows
split_index = int(total_rows * 0.9)  # 90% of data for training

train_dataset = tokenized_datasets["train"].select(range(split_index))  # First 90% rows
validation_dataset = tokenized_datasets["train"].select(range(split_index, total_rows))  # Last 10% rows

# ✅ Create `lm_datasets` preserving row structure
lm_datasets = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})

# ✅ Check if `lm_datasets` is correctly processed
print(lm_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 413
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 46
    })
})


And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [18]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

And some `TrainingArguments`:

In [19]:
from transformers import Trainer, TrainingArguments

In [20]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# -------------------------------
# 1. Setup LoRA (PEFT) for your model
# -------------------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "down_proj", "up_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap your base model with LoRA adapters
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Optional: check which parameters will be trained

# -------------------------------
# 2. Define Training Arguments
# -------------------------------
training_args = TrainingArguments(
    output_dir="deepseek-r1-finetuned-stockstimate",  # Output directory for checkpoints
    evaluation_strategy="epoch",                       # Evaluate at the end of each epoch
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    per_device_train_batch_size=1,      # Lower batch size to reduce VRAM usage
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,        # Accumulate gradients to simulate a larger batch size
    num_train_epochs=3,
    fp16=True                           # Enable mixed precision training
)

# -------------------------------
# 3. Create a Dynamic Data Collator
# -------------------------------
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Disable masked language modeling (causal LM)
)

# -------------------------------
# 4. Initialize the Trainer
# -------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],   # Use the sequentially split "test" data for evaluation
    tokenizer=tokenizer,
    data_collator=data_collator
)

# -------------------------------
# 5. Start Fine-Tuning
# -------------------------------
trainer.train()


trainable params: 9,232,384 || all params: 1,786,320,384 || trainable%: 0.5168


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-71ffffe43600>:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yusufgencer99 (yusufgencer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,4.847838
2,No log,4.450434


TrainOutput(global_step=153, training_loss=4.98684672436683, metrics={'train_runtime': 361.9867, 'train_samples_per_second': 3.423, 'train_steps_per_second': 0.423, 'total_flos': 1.2383644138758144e+16, 'train_loss': 4.98684672436683, 'epoch': 2.9491525423728815})

And we can train our model:

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [21]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 85.66


You can now upload the result of the training to the Hub, just execute this instruction:

In [22]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1738497020.1a20581f1758.555.0:   0%|          | 0.00/6.39k [00:00<?, ?B/s]

events.out.tfevents.1738497518.1a20581f1758.555.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ysfgncr/deepseek-r1-finetuned-stockstimate/commit/da42a854c54e6fc11f012191aed7b46d76c409a5', commit_message='End of training', commit_description='', oid='da42a854c54e6fc11f012191aed7b46d76c409a5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ysfgncr/deepseek-r1-finetuned-stockstimate', endpoint='https://huggingface.co', repo_type='model', repo_id='ysfgncr/deepseek-r1-finetuned-stockstimate'), pr_revision=None, pr_num=None)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -------------------------------
# 1. Load the Fine-Tuned Model and Tokenizer
# -------------------------------
model_dir = "deepseek-r1-finetuned-stockstimate"

# Load the base model (the same checkpoint you fine-tuned from)
base_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# If you fine-tuned using LoRA/PEFT, load the adapter:
model = PeftModel.from_pretrained(base_model, model_dir)

{"text": "User: Here is stock data for AAPL. Last 10 days price changes: [-0.48, 1.97, -4.04, 0.75, -3.19, 0.53, -0.08, -0.39, 3.18, 3.65]%. Volume changes: [-20.54, 1.01, 80.15, -4.56, 43.19, -34.61, -6.07, -9.19, 73.43, -20.19]%. RSI values: [19.29, 27.31, 16.75, 18.38, 16.94, 20.22, 20.93, 23.88, 35.61, 43.72]. MFI values: [30.77, 30.29, 24.23, 18.52, 17.34, 24.89, 31.89, 32.15, 41.51, 43.54]. Recent news: [{'headline': 'Apple Inc. (AAPL)\u2019s Struggle: Can AI Help Regain Market Share \u2026', 'days_ago': 14, 'snippet': 'Jan 14, 2025\\xa0\u00b7 In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against other important AI news and ratings on investors\u2019 radar. On Monday, the US \u2026'}, {'headline': \"Apple (AAPL) Laps the Stock Market: Here's Why - Yahoo Finance\", 'days_ago': 13, 'snippet': \"Jan 15, 2025\\xa0\u00b7 Apple (AAPL) closed at $237.87 in the latest trading session, marking a +1.97% move from the prior day. The stock's performance was ahead of the S&P 500's daily gain of \u2026\"}, {'headline': 'Why Apple (AAPL) Shares Are Falling Today By Stock Story', 'days_ago': 12, 'snippet': 'Jan 16, 2025\\xa0\u00b7 Shares of iPhone and iPad maker Apple (NASDAQ: AAPL) fell 3.2% in the afternoon session after data from research firm Canalys revealed the company (AAPL) is \u2026'}, {'headline': 'Company News for Jan 17, 2025 - Yahoo Finance', 'days_ago': 11, 'snippet': \"Jan 17, 2025\\xa0\u00b7 Apple Inc. (AAPL) shares fell 4% after Canalys data revealed it was overtaken by Vivo and Huawei as China's top smartphone seller in 2024. First Horizon Corporation (FHN) \u2026\"}, {'headline': 'Why Apple (AAPL) Shares Are Trading Lower Today - Yahoo \u2026', 'days_ago': 7, 'snippet': \"Jan 21, 2025\\xa0\u00b7 Shares of iPhone and iPad maker Apple (NASDAQ:AAPL) fell 4.1% in the morning session after Jefferies analyst Edison Lee downgraded the stock's rating from Hold to Sell, and \u2026\"}, {'headline': 'Is Apple Inc. (AAPL) the Best Cash App Stock to Buy According \u2026', 'days_ago': 6, 'snippet': 'Jan 22, 2025\\xa0\u00b7 Premium News. Earnings. Tech. Housing. Crypto. ... (AAPL) the Best Cash App Stock to Buy According to Hedge Funds? Fatima Gulzar . Wed, Jan 22, 2025, 1:44 PM 7 min \u2026'}, {'headline': '6 Investing Experts Predict How High Apple Stock Will Go in 2025', 'days_ago': 5, 'snippet': \"Jan 23, 2025\\xa0\u00b7 Apple's stock price closed at $222.64 per share on Jan. 21, 2024, down from its 52-week high of $260.10. Analysts have set the one-year target for Apple stock (AAPL) at \u2026\"}, {'headline': \"Apple's Earnings Strength Keeps BofA Bullish Despite iPhone \u2026\", 'days_ago': 4, 'snippet': 'Jan 24, 2025\\xa0\u00b7 Stocks: Most Actives. Stocks: Gainers. Stocks: Losers ... Jan 24, 2025, 10:08 AM 1 min read. In This Article: ... AAPL . Apple (NASDAQ:AAPL) remains a Buy at BofA Securities, \u2026'}]. What is the expected price change in 3 days?\n\nAssistant: <think>Analyzing AAPL: Over the last 10 days, the price trend has been upward. Volume has been increasing. RSI is 43.72, indicating neutral conditions. MFI is 43.54, suggesting neutral market behavior. Recent news headlines: 'Apple Inc. (AAPL)\u2019s Struggle: Can AI Help Regain Market Share \u2026' - Jan 14, 2025\u00a0\u00b7 In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against other important AI news and ratings on investors\u2019 radar. On Monday, the US \u2026; 'Apple (AAPL) Laps the Stock Market: Here's Why - Yahoo Finance' - Jan 15, 2025\u00a0\u00b7 Apple (AAPL) closed at $237.87 in the latest trading session, marking a +1.97% move from the prior day. The stock's performance was ahead of the S&P 500's daily gain of \u2026; 'Why Apple (AAPL) Shares Are Falling Today By Stock Story' - Jan 16, 2025\u00a0\u00b7 Shares of iPhone and iPad maker Apple (NASDAQ: AAPL) fell 3.2% in the afternoon session after data from research firm Canalys revealed the company (AAPL) is \u2026; 'Company News for Jan 17, 2025 - Yahoo Finance' - Jan 17, 2025\u00a0\u00b7 Apple Inc. (AAPL) shares fell 4% after Canalys data revealed it was overtaken by Vivo and Huawei as China's top smartphone seller in 2024. First Horizon Corporation (FHN) \u2026; 'Why Apple (AAPL) Shares Are Trading Lower Today - Yahoo \u2026' - Jan 21, 2025\u00a0\u00b7 Shares of iPhone and iPad maker Apple (NASDAQ:AAPL) fell 4.1% in the morning session after Jefferies analyst Edison Lee downgraded the stock's rating from Hold to Sell, and \u2026; 'Is Apple Inc. (AAPL) the Best Cash App Stock to Buy According \u2026' - Jan 22, 2025\u00a0\u00b7 Premium News. Earnings. Tech. Housing. Crypto. ... (AAPL) the Best Cash App Stock to Buy According to Hedge Funds? Fatima Gulzar . Wed, Jan 22, 2025, 1:44 PM 7 min \u2026; '6 Investing Experts Predict How High Apple Stock Will Go in 2025' - Jan 23, 2025\u00a0\u00b7 Apple's stock price closed at $222.64 per share on Jan. 21, 2024, down from its 52-week high of $260.10. Analysts have set the one-year target for Apple stock (AAPL) at \u2026; 'Apple's Earnings Strength Keeps BofA Bullish Despite iPhone \u2026' - Jan 24, 2025\u00a0\u00b7 Stocks: Most Actives. Stocks: Gainers. Stocks: Losers ... Jan 24, 2025, 10:08 AM 1 min read. In This Article: ... AAPL . Apple (NASDAQ:AAPL) remains a Buy at BofA Securities, \u2026. Considering these factors, my prediction is: </think> The stock price is expected to change by -0.95% in 3 days."}

# -------------------------------
# 2. Prepare an Inference Prompt
# -------------------------------
prompt = (
    "User: Here is stock data for AAPL. Last 10 days price changes: [-0.48, 1.97, -4.04, 0.75, -3.19, 0.53, -0.08, -0.39, 3.18, 3.65]%. Volume changes: [-20.54, 1.01, 80.15, -4.56, 43.19, -34.61, -6.07, -9.19, 73.43, -20.19]%. RSI values: [19.29, 27.31, 16.75, 18.38, 16.94, 20.22, 20.93, 23.88, 35.61, 43.72]. MFI values: [30.77, 30.29, 24.23, 18.52, 17.34, 24.89, 31.89, 32.15, 41.51, 43.54]. Recent news: [{'headline': 'Apple Inc. (AAPL)\u2019s Struggle: Can AI Help Regain Market Share \u2026', 'days_ago': 14, 'snippet': 'Jan 14, 2025\\xa0\u00b7 In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against other important AI news and ratings on investors\u2019 radar. On Monday, the US \u2026'}, {'headline': \"Apple (AAPL) Laps the Stock Market: Here's Why - Yahoo Finance\", 'days_ago': 13, 'snippet': \"Jan 15, 2025\\xa0\u00b7 Apple (AAPL) closed at $237.87 in the latest trading session, marking a +1.97% move from the prior day. The stock's performance was ahead of the S&P 500's daily gain of \u2026\"}, {'headline': 'Why Apple (AAPL) Shares Are Falling Today By Stock Story', 'days_ago': 12, 'snippet': 'Jan 16, 2025\\xa0\u00b7 Shares of iPhone and iPad maker Apple (NASDAQ: AAPL) fell 3.2% in the afternoon session after data from research firm Canalys revealed the company (AAPL) is \u2026'}, {'headline': 'Company News for Jan 17, 2025 - Yahoo Finance', 'days_ago': 11, 'snippet': \"Jan 17, 2025\\xa0\u00b7 Apple Inc. (AAPL) shares fell 4% after Canalys data revealed it was overtaken by Vivo and Huawei as China's top smartphone seller in 2024. First Horizon Corporation (FHN) \u2026\"}, {'headline': 'Why Apple (AAPL) Shares Are Trading Lower Today - Yahoo \u2026', 'days_ago': 7, 'snippet': \"Jan 21, 2025\\xa0\u00b7 Shares of iPhone and iPad maker Apple (NASDAQ:AAPL) fell 4.1% in the morning session after Jefferies analyst Edison Lee downgraded the stock's rating from Hold to Sell, and \u2026\"}, {'headline': 'Is Apple Inc. (AAPL) the Best Cash App Stock to Buy According \u2026', 'days_ago': 6, 'snippet': 'Jan 22, 2025\\xa0\u00b7 Premium News. Earnings. Tech. Housing. Crypto. ... (AAPL) the Best Cash App Stock to Buy According to Hedge Funds? Fatima Gulzar . Wed, Jan 22, 2025, 1:44 PM 7 min \u2026'}, {'headline': '6 Investing Experts Predict How High Apple Stock Will Go in 2025', 'days_ago': 5, 'snippet': \"Jan 23, 2025\\xa0\u00b7 Apple's stock price closed at $222.64 per share on Jan. 21, 2024, down from its 52-week high of $260.10. Analysts have set the one-year target for Apple stock (AAPL) at \u2026\"}, {'headline': \"Apple's Earnings Strength Keeps BofA Bullish Despite iPhone \u2026\", 'days_ago': 4, 'snippet': 'Jan 24, 2025\\xa0\u00b7 Stocks: Most Actives. Stocks: Gainers. Stocks: Losers ... Jan 24, 2025, 10:08 AM 1 min read. In This Article: ... AAPL . Apple (NASDAQ:AAPL) remains a Buy at BofA Securities, \u2026'}]. What is the expected price change in 3 days?\n\nAssistant: <think>"
)

# -------------------------------
# 3. Tokenize the Prompt and Generate a Response
# -------------------------------
# Tokenize the prompt (without fixed padding; inference will use dynamic length)
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate output with the fine-tuned model
# Adjust max_new_tokens, temperature, etc., as desired.
output_ids = model.generate(
    inputs.input_ids,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7
)

# Decode the generated tokens into text
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated Response:")
print(output_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Generated Response:
User: Here is stock data for AAPL. Last 10 days price changes: [-0.48, 1.97, -4.04, 0.75, -3.19, 0.53, -0.08, -0.39, 3.18, 3.65]%. Volume changes: [-20.54, 1.01, 80.15, -4.56, 43.19, -34.61, -6.07, -9.19, 73.43, -20.19]%. RSI values: [19.29, 27.31, 16.75, 18.38, 16.94, 20.22, 20.93, 23.88, 35.61, 43.72]. MFI values: [30.77, 30.29, 24.23, 18.52, 17.34, 24.89, 31.89, 32.15, 41.51, 43.54]. Recent news: [{'headline': 'Apple Inc. (AAPL)’s Struggle: Can AI Help Regain Market Share …', 'days_ago': 14, 'snippet': 'Jan 14, 2025\xa0· In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against other important AI news and ratings on investors’ radar. On Monday, the US …'}, {'headline': "Apple (AAPL) Laps the Stock Market: Here's Why - Yahoo Finance", 'days_ago': 13, 'snippet': "Jan 15, 2025\xa0· Apple (AAPL) closed at $237.87 in the latest trading session, marking a +1.97% move from the prior day. The stock's performance was ahead of the S&P 50